# Coupon Acceptance Analysis

## Objective 
Understand the factors that influence a customer's decision to accept or reject a driving coupon. 

## Key Questions

1. What is the overall acceptance rate for each coupon type?
2. Which customer attributes differ most between acceptance and rejection?
3. Which driving context variables differ most between acceptance and rejection?
4. What actionable recommendations follow from these differences?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for all plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Load the coupon dataset
coupon_data = pd.read_csv("data/coupons.csv")

# Calculate acceptance rate by coupon type
acceptance_by_coupon = (coupon_data
    .groupby("coupon")["Y"]
    .mean()
    .sort_values(ascending=True) * 100)

# Create bar chart
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(acceptance_by_coupon.index, acceptance_by_coupon.values, color=['#e74c3c', '#e67e22', '#f39c12', '#27ae60', '#2ecc71'])
ax.set_xlabel('Acceptance Rate (%)', fontsize=12)
ax.set_title('Coupon Acceptance Rate by Type', fontsize=14, fontweight='bold')
ax.set_xlim(0, 100)

# Add percentage labels on bars
for bar, val in zip(bars, acceptance_by_coupon.values):
    ax.text(val + 1, bar.get_y() + bar.get_height()/2, f'{val:.1f}%', va='center', fontsize=11)

plt.tight_layout()
plt.show()

**Why this chart:** A horizontal bar chart is ideal for comparing acceptance rates across coupon categories. It allows for easy visual ranking, with the longest bars representing the highest acceptance rates. The color gradient (red to green) reinforces the performance hierarchy at a glance.

### Initial Observation

The results suggest that a driver's mindset strongly influences whether a coupon is accepted. Bar coupons had the lowest acceptance rate, which makes sense since drivers are unlikely to stop for a drink while on the road. Carry out & Take away coupons had the highest acceptance rate, indicating that drivers prefer options that allow for quick stops while continuing toward their destination.

While speed appears to be an important factor, it is not the only one. If convenience alone drove acceptance, coffee house coupons would likely have higher acceptance rates than observed. Price also does not seem to be the primary factor, as higher-priced restaurants likely involve longer sit-down experiences that are less appealing to drivers. 

**Initial hypothesis:** Quick redemption time combined with a practical food option is the main driver of coupon acceptance.

Let's dive deeper into Coffee House coupons to understand why they sit at ~50% acceptance despite being a convenient option.

## Deeper Dive: Coffee House Coupons (49.92% Acceptance)

Why do coffee house coupons have a ~50% acceptance rate when they seem like a convenient, quick option? Coffee provides caffeine for alertness while driving and often includes quick bites. 

**Questions to investigate:**
1. Do people who already go to coffee houses frequently accept more?
2. Does time of day matter? (Morning commute vs. evening?)
3. Does passenger type matter? (Alone vs. with kids?)
4. Does destination matter? (Urgent vs. no rush?)

In [ ]:
# Filter data to Coffee House coupons
coffee_house_data = coupon_data.query("coupon == 'Coffee House'")

# Analyze acceptance by destination, gender, and time
gender_time_stats = (coffee_house_data
    .groupby(["destination", "gender", "time"])["Y"]
    .agg(["count", "sum", "mean"])
    .rename(columns={"count": "total", "sum": "accepted", "mean": "acceptance_rate"})
    .sort_values("acceptance_rate", ascending=False))

gender_time_stats["acceptance_rate"] = (gender_time_stats["acceptance_rate"] * 100).round(2).astype(str) + "%"
gender_time_stats

In [ ]:
# Gender doesn't appear to be a significant factor — males and females in the same 
# destination/time bucket have similar rates. Let's look at passenger type instead.

passenger_stats = (coffee_house_data
    .groupby(["destination", "passanger", "time"])["Y"]
    .agg(["count", "sum", "mean"])
    .rename(columns={"count": "total", "sum": "accepted", "mean": "acceptance_rate"})
    .sort_values("acceptance_rate", ascending=False))

passenger_stats["acceptance_rate"] = (passenger_stats["acceptance_rate"] * 100).round(2).astype(str) + "%"
passenger_stats

In [ ]:
# Visualize Coffee House acceptance by destination and time
coffee_pivot = (coffee_house_data
    .groupby(["destination", "time"])["Y"]
    .mean()
    .unstack() * 100)

# Reorder time columns logically
time_order = ['7AM', '10AM', '2PM', '6PM', '10PM']
coffee_pivot = coffee_pivot[[col for col in time_order if col in coffee_pivot.columns]]

fig, ax = plt.subplots(figsize=(10, 5))
sns.heatmap(coffee_pivot, annot=True, fmt='.0f', cmap='RdYlGn', center=50, 
            cbar_kws={'label': 'Acceptance Rate (%)'}, ax=ax)
ax.set_title('Coffee House Acceptance Rate: Destination vs. Time of Day', fontsize=14, fontweight='bold')
ax.set_xlabel('Time of Day', fontsize=12)
ax.set_ylabel('Destination', fontsize=12)
plt.tight_layout()
plt.show()

**Why this chart:** A heatmap is the best choice when analyzing two categorical variables (destination and time) against a continuous outcome (acceptance rate). The color intensity makes patterns immediately visible — green cells show high acceptance, red cells show low acceptance. This reveals that the top-left region (No Urgent Place + morning) performs best, while the bottom-right (Home + evening) performs worst.

### Coffee House: Key Drivers

The main factors influencing coffee house coupon acceptance:

- **Destination:** "No Urgent Place" has much higher acceptance than "Home"
- **Time of day:** 10AM is best, 6PM/10PM are worst
- **Passenger + Time interaction:** Kids at 10AM = 73%, but kids at 6PM = 31%

The pattern suggests that coffee is accepted when people have leisure time, not when they're in a hurry to get somewhere.

## Comparison: Carry Out & Take Away (73.55% Acceptance)

Let's compare Coffee House to Carry Out to understand why there's a 24-point difference in acceptance rates.

In [ ]:
# Filter data to Carry out & Take away coupons
carryout_data = coupon_data.query("coupon == 'Carry out & Take away'")

# Calculate stats by destination, passenger, time
carryout_stats = (carryout_data
    .groupby(["destination", "passanger", "time"])["Y"]
    .agg(["count", "sum", "mean"])
    .rename(columns={"count": "total", "sum": "accepted", "mean": "acceptance_rate"})
    .sort_values("acceptance_rate", ascending=False))

carryout_stats["acceptance_rate"] = (carryout_stats["acceptance_rate"] * 100).round(2).astype(str) + "%"
carryout_stats

## Key Finding: Want vs. Need

The comparison reveals a fundamental difference between these coupon types.

### Same Scenarios, Different Results

| Scenario | Carry Out | Coffee House |
|----------|-----------|--------------|
| Home + Alone + 6PM | 82% | 37% |
| Home + Alone + 10PM | 79% | 28% |
| No Urgent Place + Kids + 6PM | 82% | 31% |

### The Insight

**Carry out acceptance stays high regardless of destination or time** because food is a practical need. People heading home at 6PM are thinking "I need to feed myself/my family" — carry out solves that problem.

**Coffee is a want, not a need.** When you're heading home at 6PM, you don't *need* coffee. But if you're just cruising with no urgent place to be at 10AM? Sure, coffee sounds nice.

This confirms the initial hypothesis: coupons for practical necessities (food) outperform coupons for discretionary items (coffee) across all contexts.

In [ ]:
# Side-by-side comparison: Coffee House vs. Carry Out acceptance by destination and time
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Coffee House heatmap
coffee_pivot = (coffee_house_data.groupby(["destination", "time"])["Y"].mean().unstack() * 100)
time_order = ['7AM', '10AM', '2PM', '6PM', '10PM']
coffee_pivot = coffee_pivot[[col for col in time_order if col in coffee_pivot.columns]]

sns.heatmap(coffee_pivot, annot=True, fmt='.0f', cmap='RdYlGn', center=50, 
            vmin=20, vmax=90, cbar_kws={'label': '%'}, ax=axes[0])
axes[0].set_title('Coffee House (50% overall)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Time of Day')
axes[0].set_ylabel('Destination')

# Carry Out heatmap
carryout_pivot = (carryout_data.groupby(["destination", "time"])["Y"].mean().unstack() * 100)
carryout_pivot = carryout_pivot[[col for col in time_order if col in carryout_pivot.columns]]

sns.heatmap(carryout_pivot, annot=True, fmt='.0f', cmap='RdYlGn', center=50,
            vmin=20, vmax=90, cbar_kws={'label': '%'}, ax=axes[1])
axes[1].set_title('Carry Out (74% overall)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Time of Day')
axes[1].set_ylabel('')

plt.suptitle('Acceptance Rate Comparison: Want (Coffee) vs. Need (Food)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

**Why this chart:** Side-by-side heatmaps using the same color scale enable direct comparison between two coupon types. This layout makes the "Want vs. Need" insight immediately obvious — Coffee House shows dramatic variation (mix of red and green), while Carry Out is predominantly green across all scenarios. The visual contrast proves that food coupons are context-independent while coffee coupons are highly context-sensitive.

## Final Check: Do Frequent Coffee Drinkers Accept More?

In [ ]:
# Analyze acceptance by destination, time, and coffee house visit frequency
frequency_stats = (coffee_house_data
    .groupby(["destination", "time", "CoffeeHouse"])["Y"]
    .agg(["count", "sum", "mean"])
    .rename(columns={"count": "total", "sum": "accepted", "mean": "acceptance_rate"})
    .sort_values("acceptance_rate", ascending=False))

frequency_stats["acceptance_rate"] = (frequency_stats["acceptance_rate"] * 100).round(2).astype(str) + "%"
frequency_stats

### Frequency Insight

The data shows a clear pattern:
- **Frequent visitors (4+ times/month):** 65-82% acceptance in favorable conditions
- **Non-visitors ("never"):** 8-25% acceptance even in the best conditions

**Coffee coupons are only effective for existing coffee drinkers.** For people who don't already visit coffee houses, context doesn't matter; they simply won't accept.

This contrasts with carry out, which appeals universally regardless of prior behavior.

In [ ]:
# Visualize acceptance rate by coffee house visit frequency
freq_acceptance = (coffee_house_data
    .groupby("CoffeeHouse")["Y"]
    .mean()
    .sort_values(ascending=True) * 100)

# Reorder for logical display
freq_order = ['never', 'less1', '1~3', '4~8', 'gt8']
freq_acceptance = freq_acceptance.reindex([f for f in freq_order if f in freq_acceptance.index])

fig, ax = plt.subplots(figsize=(10, 5))
colors = ['#e74c3c', '#e67e22', '#f39c12', '#27ae60', '#2ecc71']
bars = ax.barh(freq_acceptance.index, freq_acceptance.values, color=colors)
ax.set_xlabel('Acceptance Rate (%)', fontsize=12)
ax.set_ylabel('Monthly Coffee House Visits', fontsize=12)
ax.set_title('Coffee Coupon Acceptance by Visit Frequency', fontsize=14, fontweight='bold')
ax.set_xlim(0, 80)

# Add percentage labels
for bar, val in zip(bars, freq_acceptance.values):
    ax.text(val + 1, bar.get_y() + bar.get_height()/2, f'{val:.1f}%', va='center', fontsize=11)

# Rename y-axis labels for clarity
ax.set_yticklabels(['Never', 'Less than 1', '1-3 times', '4-8 times', 'More than 8'])
plt.tight_layout()
plt.show()

**Why this chart:** A horizontal bar chart ordered by visit frequency shows the clear progression from non-visitors to frequent visitors. The color gradient reinforces this trend — the stark difference between "Never" (red, 19%) and "More than 8" (green, 66%) makes the targeting insight undeniable: coffee coupons only work for people who already drink coffee.

## Conclusions & Recommendations

### Key Findings

1. **Food coupons outperform beverage coupons** — Carry out (74%) and budget restaurants (71%) beat coffee houses (50%) and bars (41%)

2. **Practical needs trump discretionary wants** — Carry out acceptance stays high (65-92%) across all contexts because food is a necessity. Coffee acceptance is highly context-dependent.

3. **Coffee house coupons require the right conditions:**
   - Best: No urgent destination + mid-morning (10AM) = 65-82%
   - Worst: Heading home + evening (6PM/10PM) = 8-37%

4. **Target audience matters for coffee** — Frequent coffee drinkers accept at 65-82%, non-visitors accept at only 8-25%

### Recommendations

1. **Prioritize food-based coupons** for maximum acceptance rates across all driver contexts

2. **Time coffee house coupons strategically** — Target mid-morning (10AM) and early afternoon (2PM), avoid evening hours (6PM+)

3. **Target coffee coupons based on behavior** — Focus on users with history of coffee house visits; don't waste coupons on non-coffee drinkers

4. **Consider destination context** — Avoid sending discretionary coupons (coffee, bars) to drivers heading home; they're focused on getting there, not stopping